In [ ]:
import torch
from torch.utils import data
from d2l import torch as d2l

In [ ]:
batch_size: int = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)

num_input: int = 28 * 28
num_output: int = 10

W: torch.Tensor = torch.normal(0, 0.01, size=(num_input, num_output), requires_grad=True)
b: torch.Tensor = torch.zeros(num_output, requires_grad=True)

In [ ]:
def softmax(X: torch.Tensor) -> torch.Tensor:
    exp_sum: torch.Tensor = torch.exp(X).sum(1, keepdim=True)
    return torch.exp(X) / exp_sum

softmax指的是将输出的结果转化为概率值，即将输出的结果转化为0-1之间的值，且所有输出的值之和为1。这样做的好处是可以将输出的结果转化为概率值，从而可以更好的评估模型的好坏。
如果是对一个矩阵做softmax，那么就是对矩阵的每一行做softmax，即对每一个样本做softmax。

In [ ]:
def net(X: torch.Tensor) -> torch.Tensor:
    return softmax(X.reshape(-1, 1) @ W + b)

交叉熵损失函数，这是一个非常常用的损失函数，它的公式如下：
$H(p,q)=-\sum_{i=1}^n p_i \log q_i$
其中，p是真实的概率分布，q是预测的概率分布。交叉熵损失函数的值越小，说明预测的概率分布q越接近真实的概率分布p，模型的效果越好。在这里，真实的概率分布p是one-hot编码，即只有一个位置的值为1，其他位置的值为0，而预测的概率分布q是softmax的输出，即每个位置的值都在0-1之间，且所有位置的值之和为1。

In [ ]:
y = torch.tensor([0, 2])
y_hat = torch.tensor([[0.1, 0.3, 0.6], [0.3, 0.2, 0.5]])
y_hat[[0, 1], y]

def loss(y_hat: torch.Tensor, y: torch.Tensor) -> torch.Tensor:
    return -torch.log(y_hat[range(0, len(y_hat)), y]) # 看不懂的话可以看上面那个demo

In [ ]:
def accuracy(y_hat: torch.Tensor, y: torch.Tensor, percentage: bool=True) -> torch.Tensor:
    if len(y_hat.shape[0]) > 1 and y_hat.shape[1] > 1:
        y_hat = y_hat.argmax(dim=1)
    cmp: torch.Tensor = (y_hat.type(y.dtype) == y)
    return float(cmp.type(y.dtype).sum()) if percentage else float(cmp.type(y.dtype).sum()) / len(cmp)

In [ ]:
class Accumulator:
    data: list = None
    size: int = None
    
    def __init__(self, n: int) -> None:
        self.size = n
        self.reset()

    def add(self, *args) -> None:
        self.data = [a + float(b) for a, b in zip(self.data, args)] # 这里的意思是把两个list的元素对应相加
    
    def reset(self) -> None:
        self.data = [0.0] * self.size
    
    def __getitem__(self, index: int) -> float:
        return self.data[index]

In [ ]:
def evaluate_accuracy(net, data_iter: data.DataLoader) -> float:
    if isinstance(net, torch.nn.Module):
        net.eval()
    metric = Accumulator(2)
    with torch.no_grad():
        for X, y in data_iter:
            metric.add(accuracy(net(X), y), y.numel())
    return metric[0] / metric[1]

In [14]:
evaluate_accuracy(net, test_iter)

KeyboardInterrupt: 